# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Cookbook-for-cantera_tools-module" data-toc-modified-id="Cookbook-for-cantera_tools-module-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cookbook for cantera_tools module</a></div><div class="lev2 toc-item"><a href="#running-a-simple-simulation" data-toc-modified-id="running-a-simple-simulation-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>running a simple simulation</a></div><div class="lev2 toc-item"><a href="#reducing-a-mechanism-by-reactions" data-toc-modified-id="reducing-a-mechanism-by-reactions-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>reducing a mechanism by reactions</a></div><div class="lev2 toc-item"><a href="#analyzing-data" data-toc-modified-id="analyzing-data-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>analyzing data</a></div><div class="lev3 toc-item"><a href="#obtaining-reaction-and-species-data" data-toc-modified-id="obtaining-reaction-and-species-data-131"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>obtaining reaction and species data</a></div><div class="lev3 toc-item"><a href="#finding-stoichiometric-coefficients" data-toc-modified-id="finding-stoichiometric-coefficients-132"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>finding stoichiometric coefficients</a></div><div class="lev2 toc-item"><a href="#analyzing-data" data-toc-modified-id="analyzing-data-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>analyzing data</a></div><div class="lev3 toc-item"><a href="#finding-stoichiometric-coefficients" data-toc-modified-id="finding-stoichiometric-coefficients-141"><span class="toc-item-num">1.4.1&nbsp;&nbsp;</span>finding stoichiometric coefficients</a></div>

# Cookbook for cantera_tools module

This notebook describes some of the methods in this package and how they can be used.

In [1]:
import cantera_tools as ctt
import numpy as np
from scipy import integrate
import cantera as ct
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [16]:
import importlib
importlib.reload(ctt)

<module 'cantera_tools' from '/home/mark/workspace/tools/cantera_tools.py'>

## running a simple simulation

In [12]:
model_link = 'cookbook_files/model.cti'

# creates the cantera Solution object
solution = ctt.create_mechanism(model_link)

# finds initial mole fraction for a fuel-air ratio of 1 with 30%/70% methanol/propane blend
# for non-combustion conditions, this can be replaced by a dictionary of values {'CH3OH': 0.3, 'C3H8':0.7}
mole_fractions = ctt.get_initial_mole_fractions(stoich_ratio = 1,
                                               fuel_mole_ratios = [.3,.7],
                                               oxygen_per_fuel_at_stoich_list = [1.5,5],
                                               fuels = ['CH3OH','C3H8'])

# set initial conditions of solution in kelvin pascals and mole fractions
solution.TPX = 750, 10**6, mole_fractions

# setup adiabatic constant volume reactor
reactor = ct.IdealGasReactor(solution)
simulator = ct.ReactorNet([reactor])
solution = reactor.kinetics

# setup data storage using functions from this class - this saves states, species and reaction profiles
df = pd.DataFrame()
df = ctt.append_data_to_df(simulator, solution, df, add_rxns=True)

# run simulation
time_final = 50 #seconds
while simulator.time < time_final:
    simulator.step(time_final)
    # appends data to DataFrame
    df = ctt.append_data_to_df(simulator,solution,df, add_rxns=True)


# data can be saved to avoid rerunning the simulation for data analysis (in most cases)
df.to_pickle('cookbook_files/%s.pic' % (mole_fractions))

## reducing a mechanism by reactions
The modules can create a reduced mechanism given a list of desired reaction strings, using how cantera represents the reaction strings (this can be found by `solution.reaction_equations()`). It will remove any unused species as well.

In [23]:
desired_reactions = ['CH3OH + O2 <=> CH2OH(29) + HO2(12)',
 'C3H8 + O2 <=> C3H7(61) + HO2(12)',
 'C3H8 + O2 <=> C3H7(60) + HO2(12)',
 'CH3OH + OH(10) <=> CH2OH(29) + H2O(11)',
 'C3H8 + OH(10) <=> C3H7(60) + H2O(11)',
 'C3H8 + OH(10) <=> C3H7(61) + H2O(11)',
 'CH3OH + HO2(12) <=> CH2OH(29) + H2O2(13)',
 'C3H8 + HO2(12) <=> C3H7(61) + H2O2(13)',
 'C3H8 + HO2(12) <=> C3H7(60) + H2O2(13)',
 'C3H7(60) + O2 <=> C3H7O2(78)',
 'C3H7(61) + O2 <=> C3H7O2(80)',]

# make the reduced mechanism using the full mechanism `.cti` file.
solution_reduced = ctt.create_mechanism(model_link, kept_reaction_equations=desired_reactions)

# NOTE: this cantera Solution object can now be used like any other

## analyzing data
### obtaining reaction and species data

In [10]:
# this outputs a dataframe of just species
data_species = ctt.find_species(df)

# this outputs a dataframe of just reactions
data_reactions = ctt.find_reactions(df)

# this outputs a dataframe with just reactions involving methanol
reactions_with_methanol = ctt.find_reactions(df,species = 'CH3OH')

### finding stoichiometric coefficients

In [8]:
# this is wrapper function that outputs the stoichiometric coefficient given reaction strings:
ctt.obtain_stoichiometry_of_species(solution,'CH2OH(29)','CH3OH + HO2(12) <=> CH2OH(29) + H2O2(13)')

1.0

In [27]:
# methods can also deal with a list of reactions
reactions_with_species = ctt.find_reactions(df,'CH2OH(29)')
ctt.obtain_stoichiometry_of_species(solution,'CH2OH(29)',reactions_with_species)

Exception: No reactions found for species CH2OH(29)

## analyzing data
### finding stoichiometric coefficients

In [15]:
df = pd.DataFrame()
pd.MultiIndex.from_product([['forward'],['a','b','c']],names = ['direction, reaction'])

MultiIndex(levels=[['forward'], ['a', 'b', 'c']],
           labels=[[0, 0, 0], [0, 1, 2]])

In [19]:
df = pd.DataFrame()
df = ctt.append_forward_and_reverse_reactions_to_dataframe(solution,df)

In [22]:
s = pd.Series()
s['apple'] = 'r'

In [21]:
print(df.to_csv())

,"('forward', '2 C3H7O2(78) <=> 2 C3H7O(76) + O2')","('forward', '2 C3H7O2(80) <=> 2 C3H7O(77) + O2')","('forward', '2 HO2(12) <=> H2O2(13) + O2')","('forward', 'C2H4O(44) + CH3(25) <=> C3H7O(77)')","('forward', 'C2H5(39) + CH2O(17) <=> C3H7O(76)')","('forward', 'C2H5(39) + O2 (+M) <=> C2H4(38) + HO2(12) (+M)')","('forward', 'C2H5(39) + O2 (+M) <=> C2H5O2(50) (+M)')","('forward', 'C2H5O2(50) (+M) <=> C2H4(38) + HO2(12) (+M)')","('forward', 'C2H5O2(50) + C3H8 <=> C2H6O2(51) + C3H7(60)')","('forward', 'C2H5O2(50) + C3H8 <=> C2H6O2(51) + C3H7(61)')","('forward', 'C2H5O2(50) + CH3OH <=> C2H6O2(51) + CH2OH(29)')","('forward', 'C2H5O2(50) + HO2(12) <=> C2H6O2(51) + O2')","('forward', 'C2H6O2(51) <=> OH(10) + ethoxy(49)')","('forward', 'C3H6(59) + OH(10) <=> H2O(11) + allyl(63)')","('forward', 'C3H6O2(95) <=> C3H5O(72) + OH(10)')","('forward', 'C3H6O3(94) <=> OH(10) + S(101)')","('forward', 'C3H7(60) + O2 <=> C3H6(59) + HO2(12)')","('forward', 'C3H7(60) + O2 <=> C3H7O2(78)')","('forward', 'C3